In [1]:
import os
import tensorflow as tf
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

from dataset import get_csi_pipeline, get_val_csi
from jr2net.utils import dd_cassi , dd_icassi, coded2DTO3D

In [2]:
RGB = [27, 17 , 4]
BATCH_SIZE = 3
PATCHES = 9       # The final batch size is BATCH_SIZE * PATCHES
split = 0.9
INPUT_SHAPE = (104, 104, 31)
FOLDER_PATH = "G:\Mi unidad\hdsp\data"

data_path = os.path.join(FOLDER_PATH, 'kaist_dataset', 'train')
test_path = data_path.replace('train', 'test')

DATA__PARAMS = dict(
    input_size = INPUT_SHAPE,
    batch_size = BATCH_SIZE,
)


train_ds = get_csi_pipeline(data_path=data_path, buffer_size=10, **DATA__PARAMS, factor=PATCHES)
# val_ds = get_csi_pipeline(data_path=test_path, buffer_size=None, **DATA__PARAMS)
val_ds = get_val_csi(data_path)

In [3]:
from gc import callbacks
from jr2net.models import JR2net
from jr2net.metrics import prior_loss, psnr
from jr2net.utils import ClearMemory
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, ReduceLROnPlateau
from tensorflow.keras import backend as k

STAGES = 7
FACTORS = [1, 1, 1/2, 1/2, 1/4, 1/8]
EPOCHS = 10

MODEL_PARAMS = {
    'loss': ['mse', 'mse'],
    'optimizer': tf.keras.optimizers.Adam(learning_rate=1e-4, amsgrad=False),
    'metrics': [psnr]
}


UNROLLED_PARAMS = {
    'input_size' : (None, None, INPUT_SHAPE[-1]),
    'num_stages': STAGES,
    'factors': FACTORS,
    'training': True,
}

unrolled_weights = "unrolled_weights.h5"
callbacks = [ModelCheckpoint(unrolled_weights,
                             monitor='recons_psnr',
                             save_best_only=True,
                             save_weights_only=True,
                             mode='max'),
             ClearMemory(),
             ReduceLROnPlateau(monitor="recons_psnr",
                               factor=0.9,
                               patience=100,
                               verbose=0,
                               mode="max")]

main_model = JR2net(**UNROLLED_PARAMS)
model = main_model.unrolled
model.compile(**MODEL_PARAMS)
model.summary()


KerasTensor(type_spec=TensorSpec(shape=(None, None, None, 1), dtype=tf.float32, name=None), name='forward_cassi/PartitionedCall:0', description="created by layer 'forward_cassi'")
Model: "JR2net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
spectral_image (InputLayer)     [(None, None, None,  0                                            
__________________________________________________________________________________________________
coded_aperture (InputLayer)     [(None, None, None,  0                                            
__________________________________________________________________________________________________
forward_cassi (Lambda)          (None, None, None, 1 0           spectral_image[0][0]             
                                                                 coded_aperture[0][0]             
            

In [4]:
model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=None)

Epoch 1/10
      2/Unknown - 95s 44s/step - loss: 0.0320 - recons_loss: 0.0165 - repre_loss: 0.0155 - recons_psnr: 23.1495 - repre_psnr: 23.3340